In [1]:
!git clone https://github.com/shoaib6174/MultiWOZ-Clustering/


!pip install pyvis

Cloning into 'MultiWOZ-Clustering'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 105 (delta 54), reused 83 (delta 32), pack-reused 0
Receiving objects: 100% (105/105), 14.89 MiB | 6.57 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Checking out files: 100% (38/38), done.


In [2]:
import pandas as pd
import numpy as np

from pyvis.network import Network
from IPython.core.display import display, HTML

import collections
import math

In [38]:
def show_graph(df, min_weight=1):
    df_edges = df[["cluster_id", "next_utterance_cluster_id"]]
    df_edges = df_edges.dropna()
    df_edges.columns = ['Source', 'Target']
    df_edges_weights = df_edges.groupby(['Source','Target']).size().reset_index().rename(columns={0:'Weight'})
    df_edges_weights = df_edges_weights[df_edges_weights['Weight'] >= min_weight]
    print(len(df_edges_weights.index))
    dialogue_graph = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', directed=True)
    dialogue_graph.barnes_hut()

    sources = df_edges_weights['Source']
    targets = df_edges_weights['Target']
    weights = df_edges_weights['Weight']

    edge_data = zip(sources, targets, weights)



    for edge in edge_data:
        src = edge[0]
        dst = edge[1]
        w = edge[2]

        dialogue_graph.add_node(src, src, title=src)
        dialogue_graph.add_node(dst, dst, title=dst)
        dialogue_graph.add_edge(src, dst, value=w)

    neighbor_map = dialogue_graph.get_adj_list()  #dictionary mapping of Node ID to list of Node IDs 

    # add neighbor data to node hover data (Just provides list of all connected nodes to a given node
    for node in dialogue_graph.nodes:
        node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
        node['value'] = len(neighbor_map[node['id']])

    dialogue_graph.show('dialogue_graph.html')

    display(HTML("dialogue_graph.html"))

# Import utterances df with clusters and nodes df

In [30]:
df = pd.read_csv("/content/MultiWOZ-Clustering/Universal Sentence Encoder/clusters_USE_200_with_replacement.csv")
df = df.set_index("utterance_id")
df

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-SNG02022,0,SNG02022,hospital,['hospital'],0,1-SNG02022,USER,I am looking for the Addenbrookes Hospital with hepatology department,i am looking for the addenbrookes hospital with hepatology department,u-30,s-57
1-SNG02022,1,SNG02022,hospital,['hospital'],1,2-SNG02022,SYSTEM,The telephone number is 01223217712,the telephone number is 01223217712,s-57,u-46
2-SNG02022,2,SNG02022,hospital,['hospital'],2,3-SNG02022,USER,Thank you Goodbye,thank you goodbye,u-46,s-45
3-SNG02022,3,SNG02022,hospital,['hospital'],3,NaN,SYSTEM,Have a wonderful day!,have a wonderful day,s-45,NaN
0-SNG01863,4,SNG01863,taxi,['taxi'],0,1-SNG01863,USER,I'm at Kettle's Yard and need a taxi to pick me up by 19:30.,i m at kettle s yard and need a taxi to pick me up by 1930,u-9,s-9
...,...,...,...,...,...,...,...,...,...,...,...
15-MUL2662,33277,MUL2662,attraction & hotel,"['attraction', 'hotel']",15,16-MUL2662,SYSTEM,"Yes, several. The ADC Theatre, the Cambridge Arts Theatre, Mumford Theatre, and The Cambridge Corn Exchange. Would you like to hear more about any of those?",yes several the adc theatre the cambridge arts theatre mumford theatre and the cambridge corn exchange would you like to hear more about any of those,s-62,u-56
16-MUL2662,33278,MUL2662,attraction & hotel,"['attraction', 'hotel']",16,17-MUL2662,USER,"Yes, Can I have the post code and entrance fee for the ADC?",yes can i have the post code and entrance fee for the adc,u-56,s-61
17-MUL2662,33279,MUL2662,attraction & hotel,"['attraction', 'hotel']",17,18-MUL2662,SYSTEM,"Postal is cb58as, I do not have inform on the entrance fee, you may need to call them at 01223300085",postal is cb58as i do not have inform on the entrance fee you may need to call them at 01223300085,s-61,u-47


In [31]:
pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_rows', 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [32]:
nodes_df = pd.read_csv("/content/MultiWOZ-Clustering/Universal Sentence Encoder/nodes_USE_with_10_utterances.csv")
nodes_df.columns = ["cluster_id", "10_utterances"]

nodes_df.head(10)

,cluster_id,10_utterances
0,u-0,"['thank you i also need a swimming pool in the north area of cambridge', 'i am looking for information in cambridge', 'i am looking for information in cambridge a a place to dine', 'where is the hospital in cambridge', 'can you give me the address to the hospital in cambridge', 'yes can i have more information about the cambridge museum of tech what is their address', 'i need to find a british restaurant in west cambridge', 'i am planning a trip in cambridge looking for a place to dine', 'i am looking for information in cambridge', 'could you help me find the cambridge belfry']"
1,u-1,"['yes how long is the travel time', 'that works can i have the travel time for that please', 'can you tell me the duration of travel time please', 'not right now what s the travel time on that ride', 'can you also tell me the travel time', 'could you tell me the travel time please first', 'can you tell me the arrival time of that train', 'yes can i have the train id and travel time please', 'it ll only be me can i get the price train id and departure time as well', 'can i get the departure and travel time of the train arriving closest to 1530']"
2,u-2,"['the train will be leaving cambridge and going to london liverpool street', 'i want to leave from london liverpool street', 'i would also like a train departing from london liverpool street and arriving in cambridge at 0900 on the same day', 'i need a train from london liverpool street station on wednesday', 'i need to depart from london liverpool street after 1515 can you tell me how long the travel time is too', 'i do need a train from london liverpool street on tuesday please', 'i ll will be leaving from london liverpool street', 'find me train times from cambridge to london liverpool street', 'i will be leaving cambridge and going to london i d like to use the liverpool street station', 'i d like a train to london liverpool street that leaves after 1000']"
3,u-3,"['okay i m looking for a train that departs from cambridge on tuesday and arrives by 0800 in london kings cross', 'i should have told you that already i apologize i need to take a train from cambridge to london kings cross on thursday leaving after 1500', 'i need to leave on sunday and my destination is london kings cross', 'could you check one more time i m departing cambridge and going to london kings cross i should arrive by 1030 on sunday', 'first of all i need a train to get there from london kings cross can you help find one', 'i d like to depart from london kings cross going to cambridge after 0930', 'i am looking for a train departing from london kings cross', 'i need a train to leave from london s cross', 'i am looking for a train departing from london kings cross', 'i need to take a train from cambridge to london kings cross']"
4,u-4,"['i m going to be leaving birmingham new street and need to arrive by 1745', 'i ve like to go to birmingham new street', 'i am leaving from birmingham new street any time after 1315', 'i am looking for a train the train should leave on monday and should go to birmingham new street', 'leaving cambridge going to birmingham new street on monday any time after 1515', 'i m planning a trip to cambridge and need info on trains i am coming from birmingham new st', 'i am going to birmingham new street', 'i need to leave after 1130 and go to birmingham new street', 'i need transportation specifically a train from birmingham new street i need one that gets to my destination by 1700', 'i need to book a train from birmingham new street to cambridge']"
5,u-5,"['i would like to book one starting friday', 'i am planning to travel on thursday', 'i would like to leave on wednesday', 'i would like to leave in saturday', 'i want to do some boating today', 'i will depart on wednesday', 'i want to leave on sunday', 'it needs to be on tuesday', 'i need to go to kings lynn on thursday', 'i will be going to ely on thursday']"
6,u-6,"['i want to go to castle galleries', 'i d like to lea

#Graphs

### all Services

In [43]:
min_edge_weight = 5
show_graph(df, min_edge_weight)

1165


### Service Wise Graph

In [8]:
set(df['services'])

{'attraction',
 'attraction & hotel',
 'attraction & train',
 'hospital',
 'hotel',
 nan,
 'restaurant',
 'restaurant & attraction',
 'restaurant & hotel',
 'restaurant & taxi & attraction',
 'restaurant & taxi & hotel',
 'restaurant & train',
 'taxi',
 'taxi & attraction & hotel',
 'train',
 'train & hotel'}

attraction

In [9]:
# write the name of services from the set above to check graph
services = ['attraction']


df_services = df[df["services"].isin(services)]
df_services

,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,,,,,,,
0-PMUL2031,168,PMUL2031,attraction,['attraction'],0,1-PMUL2031,USER,Please help me find people's portraits exhibition at girton college.,please help me find people s portraits exhibition at girton college,u-35,s-59
1-PMUL2031,169,PMUL2031,attraction,['attraction'],1,2-PMUL2031,SYSTEM,People's Portraits Exhibition at Girton College is located at Girton College on Huntingdon Road in the west side of town. Would you like their phone number?,people s portraits exhibition at girton college is located at girton college on huntingdon road in the west side of town would you like their phone number,s-59,u-57
2-PMUL2031,170,PMUL2031,attraction,['attraction'],2,3-PMUL2031,USER,Please send me their address.,please send me their address,u-57,s-54
3-PMUL2031,171,PMUL2031,attraction,['attraction'],3,4-PMUL2031,SYSTEM,"Sure, the address is Girton College, Huntingdon Road cb3ojg. Is there anything else I can help you with?",sure the address is girton college huntingdon road cb3ojg is there anything else i can help you with,s-54,u-51
4-PMUL2031,172,PMUL2031,attraction,['attraction'],4,5-PMUL2031,USER,That is all I need.,that is all i need,u-51,s-63
5-PMUL2031,173,PMUL2031,attraction,['attraction'],5,NaN,SYSTEM,"Great! By the way, there is no entrance fee as well. Let us know if we can help with anything else.",great by the way there is no entrance fee as well let us know if we can help with anything else,s-63,NaN
0-SNG1058,818,SNG1058,attraction,['attraction'],0,1-SNG1058,USER,I'm looking for a place to go in town to have fun. Are there any nightclubs in the west?,i m looking for a place to go in town to have fun are there any nightclubs in the west,u-28,s-75
1-SNG1058,819,SNG1058,attraction,['attraction'],1,2-SNG1058,SYSTEM,"There are no nightclubs in the west, would another area work for you?",there are no nightclubs in the west would another area work for you,s-75,u-42
2-SNG1058,820,SNG1058,attraction,['attraction'],2,3-SNG1058,USER,How about a college in the west?,how about a college in the west,u-42,s-59


In [45]:
min_edge_weight = 2
show_graph(df_services, min_edge_weight)

141


# Repetation per cluster per dialogue

In [11]:
min_number_of_repetation_per_dialogue = 2
rep_per_cluster_per_dialogue = df.groupby(["dialogue_id", 'cluster_id'])["cluster_id"].count().to_frame().add_suffix('_Count').reset_index()
rep_per_cluster_per_dialogue = rep_per_cluster_per_dialogue[rep_per_cluster_per_dialogue["cluster_id_Count"] >= min_number_of_repetation_per_dialogue].reset_index().drop(['index'], axis=1)
rep_per_cluster_per_dialogue.columns = ["dialogue_id", "cluster_id", 'rep_count']
rep_per_cluster_per_dialogue

,dialogue_id,cluster_id,rep_count
0,MUL0005,s-1,2
1,MUL0005,u-14,2
2,MUL0005,u-9,2
3,MUL0009,u-11,3
4,MUL0009,u-16,2
...,...,...,...
3462,WOZ20630,s-23,2
3463,WOZ20634,u-37,2
3464,WOZ20636,s-23,2
3465,WOZ20670,u-25,2


counting rep_count

In [12]:
rep_per_cluster_per_dialogue.drop(['dialogue_id'], axis=1).groupby([ 'rep_count'])['cluster_id'].count().to_frame()

,cluster_id
rep_count,
2,3004
3,382
4,68
5,10
6,3


# Cluster repetition

### occurrence of cluster repitations sorted

In [13]:
min_Count_of_cluster_id = 1
occurrence_of_cluster_rep_per_cluster = rep_per_cluster_per_dialogue.groupby(['cluster_id'])['cluster_id'].count().to_frame().add_prefix('Count_of_').reset_index().sort_values(['Count_of_cluster_id'],ascending=False )
occurrence_of_cluster_rep_per_cluster.columns = ["cluster_id", 'num_of_repetation_occurances']
occurrence_of_cluster_rep_per_cluster = pd.merge(occurrence_of_cluster_rep_per_cluster, nodes_df, how='left')

occurrence_of_cluster_rep_per_cluster = occurrence_of_cluster_rep_per_cluster[occurrence_of_cluster_rep_per_cluster["num_of_repetation_occurances"] >= min_Count_of_cluster_id]
occurrence_of_cluster_rep_per_cluster.head(10)

,cluster_id,num_of_repetation_occurances,10_utterances
0,s-23,395,"['there are 3 turkish restaurants in the centre would you prefer expensive or moderate price range', 'i ve found 8 restaurant that meet your criteria would you like to narrow your search by price range', 'what price range do you want your restaurant', 'there are 15 cheap restaurants in the centre what type of food do you want to eat', 'there are 21 great places to choose from do you have a certain type of restaurant you are looking for perhaps italian or european', 'there are nt any thai restaurants in that price range in the centre of town would you like to try something else', 'i have 7 british restaurants in the centre of town which price range would you prefer', 'we have over 110 restaurants to choose from can you tell me what your pricerange andor style of food might be to narrow it down', 'certainly there are 21 moderate restaurants to choose from in the centre are you in the mood for any particular type of food', 'i have found results for 14 restaurants in the west are you looking for a particular type of cuisine or price range']"
1,u-0,383,"['thank you i also need a swimming pool in the north area of cambridge', 'i am looking for information in cambridge', 'i am looking for information in cambridge a a place to dine', 'where is the hospital in cambridge', 'can you give me the address to the hospital in cambridge', 'yes can i have more information about the cambridge museum of tech what is their address', 'i need to find a british restaurant in west cambridge', 'i am planning a trip in cambridge looking for a place to dine', 'i am looking for information in cambridge', 'could you help me find the cambridge belfry']"
2,u-11,235,"['not really can you book one of them for friday for me', 'i would like you to book please', 'that will work can you book that for me', 'that sounds perfect can you reserve it for seven people', 'could you redo booking for 3 people please', 'thank you so much would you be able to book it', 'can you book this hotel', 'just make sure i get to the locations by the booked time', 'thank you can you book it for me', 'do nt book it but can you send me their full contact info thanks']"
3,u-47,233,"['no that is all thank you for the help', 'no i do not think so thank you for your help', 'no i think you have answered all my questions thank you', 'no that was it thank you so much for your help', 'no you have helped me fully thank you very much', 'no that s all i needed today thanks for your help bye', 'no that will be all thank you very much for your help', 'no that should do it thanks for the help', 'no that s all the information i need thanks for your help', 'nope that will be all thank you very much for your time']"
4,u-25,202,"['can you help me out with finding a restaurant that serves british food in the east', 'yes i would like to find a mediterranean restaurant in the cheap price range', 'i need help finding a cheap restaurant', 'i am looking for italian restaurants in cambridge in the cheap price range', 'i would like to go to an expensive italian restaurant for dinner', 'i would like to dine somewhere that is expensive', 'i m also looking for a place to dine the restaurant should be in the cheap price range and should serve spanish food', 'i am looking for a restaurant to book a reservation i want asian oriental in the expensive price range', 'can you help me find some cheap italian food', 'i want to find a cheap italian restaurant can you help me out']"
5,u-56,182,"['can you please provide me with the address postcode and phone number', 'can you give me their phone number and postcode', 'no any you recommend will be fine i ll just need a phone number postcode and address please', 'could i get a postcode and a phone number', 'yes what is the postcode and phone number also i am needing a 4 star hotel in the same area can you check if one is available for me', 'no thanks i just need the phone number address and postcode', 'yes i would

Count of how many times a cluster has repeated in a dialogue

In [14]:
repeated_clusters = rep_per_cluster_per_dialogue.groupby(['cluster_id', "rep_count"])['cluster_id'].count().to_frame().add_suffix('_Count').reset_index()
repeated_clusters.columns = ['cluster_id', 'rep_count_in_single_dialogue', 'count']

repeated_clusters.head(10)

,cluster_id,rep_count_in_single_dialogue,count
0,s-0,2,1
1,s-1,2,24
2,s-10,2,10
3,s-10,3,1
4,s-11,2,1
5,s-13,2,14
6,s-14,2,7
7,s-15,2,5
8,s-16,2,2
9,s-17,2,89


### sorted by  rep_count_in_single_dialogue and count

Read like: {cluster id} has repeated {rep_count_in_single_dialogue} times in {count} dialogues

In [15]:
repeated_clusters_sorted = repeated_clusters.sort_values(['rep_count_in_single_dialogue', 'count'],ascending=False )
repeated_clusters_sorted = pd.merge(repeated_clusters_sorted, nodes_df, how='left')
repeated_clusters_sorted.head(5)

,cluster_id,rep_count_in_single_dialogue,count,10_utterances
0,s-23,6,2,"['there are 3 turkish restaurants in the centre would you prefer expensive or moderate price range', 'i ve found 8 restaurant that meet your criteria would you like to narrow your search by price range', 'what price range do you want your restaurant', 'there are 15 cheap restaurants in the centre what type of food do you want to eat', 'there are 21 great places to choose from do you have a certain type of restaurant you are looking for perhaps italian or european', 'there are nt any thai restaurants in that price range in the centre of town would you like to try something else', 'i have 7 british restaurants in the centre of town which price range would you prefer', 'we have over 110 restaurants to choose from can you tell me what your pricerange andor style of food might be to narrow it down', 'certainly there are 21 moderate restaurants to choose from in the centre are you in the mood for any particular type of food', 'i have found results for 14 restaurants in the west are you looking for a particular type of cuisine or price range']"
1,u-11,6,1,"['not really can you book one of them for friday for me', 'i would like you to book please', 'that will work can you book that for me', 'that sounds perfect can you reserve it for seven people', 'could you redo booking for 3 people please', 'thank you so much would you be able to book it', 'can you book this hotel', 'just make sure i get to the locations by the booked time', 'thank you can you book it for me', 'do nt book it but can you send me their full contact info thanks']"
2,s-23,5,4,"['there are 3 turkish restaurants in the centre would you prefer expensive or moderate price range', 'i ve found 8 restaurant that meet your criteria would you like to narrow your search by price range', 'what price range do you want your restaurant', 'there are 15 cheap restaurants in the centre what type of food do you want to eat', 'there are 21 great places to choose from do you have a certain type of restaurant you are looking for perhaps italian or european', 'there are nt any thai restaurants in that price range in the centre of town would you like to try something else', 'i have 7 british restaurants in the centre of town which price range would you prefer', 'we have over 110 restaurants to choose from can you tell me what your pricerange andor style of food might be to narrow it down', 'certainly there are 21 moderate restaurants to choose from in the centre are you in the mood for any particular type of food', 'i have found results for 14 restaurants in the west are you looking for a particular type of cuisine or price range']"
3,u-0,5,2,"['thank you i also need a swimming pool in the north area of cambridge', 'i am looking for information in cambridge', 'i am looking for information in cambridge a a place to dine', 'where is the hospital in cambridge', 'can you give me the address to the hospital in cambridge', 'yes can i have more information about the cambridge museum of tech what is their address', 'i need to find a british restaurant in west cambridge', 'i am planning a trip in cambridge looking for a place to dine', 'i am looking for information in cambridge', 'could you help me find the cambridge belfry']"
4,s-21,5,1,"['the booking was unsuccessful would another day or time work', 'i am sorry it appears that the hotel does not have anything available would you like to try a shorter stay or a different check in date', 'i m so sorry but there are no available rooms for that length of stay starting on sunday would you like to book a different day or shorten your stay', 'there is no availability at that time in the establishment would you like to try a different place', 'unfortunately it can not be booked for 3 days did you want to get information about a different hotel instead', 'i am sorry but i ca nt book it without the time when did you want to go', 'i am unable to book that for you we can try another day o

## dialogues sorted based on highest repetation

Read like: {dialogue_id} has {count} clusters which repeated {repetation_type} times

In [16]:
dialogue_rep = rep_per_cluster_per_dialogue.drop(['cluster_id'], axis=1).groupby([ 'dialogue_id','rep_count' ])['rep_count'].count().to_frame().add_suffix('_Count').reset_index()

dialogue_rep.columns = ["dialogue_id", "repetation_type", "count"]
dialogue_rep = dialogue_rep.sort_values(['repetation_type', 'count'],ascending=False ).reset_index()
dialogue_rep.drop(['index'],axis=1 , inplace=True)

dialogue_rep.head(5)

,dialogue_id,repetation_type,count
0,MUL0097,6,1
1,MUL1332,6,1
2,PMUL2001,6,1
3,MUL1332,5,2
4,MUL0666,5,1


#### find dialogue with ids to check the utterances

In [17]:
dia_ids_to_find = [ "MUL1332"]

see repetation count of each cluster for the selected dialogues

In [18]:
rep_count_for_selected_dia = rep_per_cluster_per_dialogue[ rep_per_cluster_per_dialogue["dialogue_id"].isin(dia_ids_to_find)]
rep_count_for_selected_dia

,dialogue_id,cluster_id,rep_count
621,MUL1332,s-21,5
622,MUL1332,s-23,6
623,MUL1332,s-69,2
624,MUL1332,u-11,5
625,MUL1332,u-23,2
626,MUL1332,u-41,4
627,MUL1332,u-47,2


Check top 10 utterances of choosen cluster

In [19]:
cluster_ids = rep_count_for_selected_dia['cluster_id']
#cluster_ids = ["s-21", "s-23", "u-11", "u-41"]

nodes_df[nodes_df['cluster_id'].isin(cluster_ids)]

,cluster_id,10_utterances
11,u-11,"['not really can you book one of them for friday for me', 'i would like you to book please', 'that will work can you book that for me', 'that sounds perfect can you reserve it for seven people', 'could you redo booking for 3 people please', 'thank you so much would you be able to book it', 'can you book this hotel', 'just make sure i get to the locations by the booked time', 'thank you can you book it for me', 'do nt book it but can you send me their full contact info thanks']"
23,u-23,"['i want to eat thai food', 'how about thai food', 'how about chinese type of food', 'are there any that have asian oriental food', 'hmmm malaysian is sounding pretty good right now', 'yes i would like the phone numbers i am also looking for a cheap asian restaurant', 'can you suggest one no wait chinese sounds wonderful i love refried rice', 'sure i like chinese', 'yes please i m also looking for an asian oriental restaurant within the cheap range can you find one for me', 'maybe one that serves chinese food and it should be located in the centre']"
41,u-41,"['i prefer the gonville', 'how about one that is in the type of cinema', 'oh that s too bad how about monday', 'anything would be fine but lets look up entertainment', 'can you try again for me', 'how about a boating place then', 'ok how about scudamores punting company then', '1145 as discussed above', 'ok how about just european any luck there', 'could you try a cheap one then']"
47,u-47,"['no that is all thank you for the help', 'no i do not think so thank you for your help', 'no i think you have answered all my questions thank you', 'no that was it thank you so much for your help', 'no you have helped me fully thank you very much', 'no that s all i needed today thanks for your help bye', 'no that will be all thank you very much for your help', 'no that should do it thanks for the help', 'no that s all the information i need thanks for your help', 'nope that will be all thank you very much for your time']"
83,s-21,"['the booking was unsuccessful would another day or time work', 'i am sorry it appears that the hotel does not have anything available would you like to try a shorter stay or a different check in date', 'i m so sorry but there are no available rooms for that length of stay starting on sunday would you like to book a different day or shorten your stay', 'there is no availability at that time in the establishment would you like to try a different place', 'unfortunately it can not be booked for 3 days did you want to get information about a different hotel instead', 'i am sorry but i ca nt book it without the time when did you want to go', 'i am unable to book that for you we can try another day or a shorter stay if you like', 'unfortunately that did nt work do you want to shorten your trip', 'i m sorry it seems they are unable to accommodate you at that day and time would you like me to try another', 'i m very sorry but there are no rooms available for five days starting on thursday should i try to find a different hotel']"
85,s-23,"['there are 3 turkish restaurants in the centre would you prefer expensive or moderate price range', 'i ve found 8 restaurant that meet your criteria would you like to narrow your search by price range', 'what price range do you want your restaurant', 'there are 15 cheap restaurants in the centre what type of food do you want to eat', 'there are 21 great places to choose from do you have a certain type of restaurant you are looking for perhaps italian or european', 'there are nt any thai restaurants in that price range in the centre of town would you like to try something else', 'i have 7 british restaurants in the centre of town which price range would you prefer', 'we have over 110 restaurants to choose from can you tell me what your pricerange andor style of food might be to narrow it down', 'certainly there are 21 moderate restaurants to choose from in the centre are you in the mood for any particular type of f

check the uttereances of selected dialogues

In [20]:
df[df["dialogue_id"].isin(dia_ids_to_find)][["dialogue_id", 'services', "text", "cluster_id", "next_utterance_cluster_id"]]

,dialogue_id,services,text,cluster_id,next_utterance_cluster_id
utterance_id,,,,,
0-MUL1332,MUL1332,restaurant & hotel,Hello! Are there any chinese restaurants in the centre?,u-23,s-69
1-MUL1332,MUL1332,restaurant & hotel,There are quite a few in the centre. What is the price range you'd like to stay within?,s-69,u-37
2-MUL1332,MUL1332,restaurant & hotel,"Cheap price range, please!",u-37,s-23
3-MUL1332,MUL1332,restaurant & hotel,jinling noodle bar is a good fit. Its on 11 Peas Hill City Centre,s-23,u-11
4-MUL1332,MUL1332,restaurant & hotel,Great! Can you book me a table for 5 on Thursday?,u-11,s-66
5-MUL1332,MUL1332,restaurant & hotel,I sure can. Any preference on time?,s-66,u-41
6-MUL1332,MUL1332,restaurant & hotel,Please try for 12:30.,u-41,s-23
7-MUL1332,MUL1332,restaurant & hotel,"When booking your restaurant, I see that the jinling noodle bar is moderately priced. Would that work?",s-23,u-11
8-MUL1332,MUL1332,restaurant & hotel,Yes let's try making a reservation for 5 people on Thursday at 12:30.,u-11,s-21


In [21]:
df = df.reset_index()
df

,utterance_id,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
0,0-SNG02022,0,SNG02022,hospital,['hospital'],0,1-SNG02022,USER,I am looking for the Addenbrookes Hospital with hepatology department,i am looking for the addenbrookes hospital with hepatology department,u-30,s-57
1,1-SNG02022,1,SNG02022,hospital,['hospital'],1,2-SNG02022,SYSTEM,The telephone number is 01223217712,the telephone number is 01223217712,s-57,u-46
2,2-SNG02022,2,SNG02022,hospital,['hospital'],2,3-SNG02022,USER,Thank you Goodbye,thank you goodbye,u-46,s-45
3,3-SNG02022,3,SNG02022,hospital,['hospital'],3,NaN,SYSTEM,Have a wonderful day!,have a wonderful day,s-45,NaN
4,0-SNG01863,4,SNG01863,taxi,['taxi'],0,1-SNG01863,USER,I'm at Kettle's Yard and need a taxi to pick me up by 19:30.,i m at kettle s yard and need a taxi to pick me up by 1930,u-9,s-9
...,...,...,...,...,...,...,...,...,...,...,...,...
33277,15-MUL2662,33277,MUL2662,attraction & hotel,"['attraction', 'hotel']",15,16-MUL2662,SYSTEM,"Yes, several. The ADC Theatre, the Cambridge Arts Theatre, Mumford Theatre, and The Cambridge Corn Exchange. Would you like to hear more about any of those?",yes several the adc theatre the cambridge arts theatre mumford theatre and the cambridge corn exchange would you like to hear more about any of those,s-62,u-56
33278,16-MUL2662,33278,MUL2662,attraction & hotel,"['attraction', 'hotel']",16,17-MUL2662,USER,"Yes, Can I have the post code and entrance fee for the ADC?",yes can i have the post code and entrance fee for the adc,u-56,s-61
33279,17-MUL2662,33279,MUL2662,attraction & hotel,"['attraction', 'hotel']",17,18-MUL2662,SYSTEM,"Postal is cb58as, I do not have inform on the entrance fee, you may need to call them at 01223300085",postal is cb58as i do not have inform on the entrance fee you may need to call them at 01223300085,s-61,u-47
33280,18-MUL2662,33280,MUL2662,attraction & hotel,"['attraction', 'hotel']",18,19-MUL2662,USER,"Ok perfect, that is all that I needed. Thank you for your help! Goodbye.",ok perfect that is all that i needed thank you for your help goodbye,u-47,s-41


In [22]:
df

,utterance_id,Unnamed: 0,dialogue_id,services,services_list,utterance_no,next_utterance_id,speaker,text,cleaned_text,cluster_id,next_utterance_cluster_id
0,0-SNG02022,0,SNG02022,hospital,['hospital'],0,1-SNG02022,USER,I am looking for the Addenbrookes Hospital with hepatology department,i am looking for the addenbrookes hospital with hepatology department,u-30,s-57
1,1-SNG02022,1,SNG02022,hospital,['hospital'],1,2-SNG02022,SYSTEM,The telephone number is 01223217712,the telephone number is 01223217712,s-57,u-46
2,2-SNG02022,2,SNG02022,hospital,['hospital'],2,3-SNG02022,USER,Thank you Goodbye,thank you goodbye,u-46,s-45
3,3-SNG02022,3,SNG02022,hospital,['hospital'],3,NaN,SYSTEM,Have a wonderful day!,have a wonderful day,s-45,NaN
4,0-SNG01863,4,SNG01863,taxi,['taxi'],0,1-SNG01863,USER,I'm at Kettle's Yard and need a taxi to pick me up by 19:30.,i m at kettle s yard and need a taxi to pick me up by 1930,u-9,s-9
...,...,...,...,...,...,...,...,...,...,...,...,...
33277,15-MUL2662,33277,MUL2662,attraction & hotel,"['attraction', 'hotel']",15,16-MUL2662,SYSTEM,"Yes, several. The ADC Theatre, the Cambridge Arts Theatre, Mumford Theatre, and The Cambridge Corn Exchange. Would you like to hear more about any of those?",yes several the adc theatre the cambridge arts theatre mumford theatre and the cambridge corn exchange would you like to hear more about any of those,s-62,u-56
33278,16-MUL2662,33278,MUL2662,attraction & hotel,"['attraction', 'hotel']",16,17-MUL2662,USER,"Yes, Can I have the post code and entrance fee for the ADC?",yes can i have the post code and entrance fee for the adc,u-56,s-61
33279,17-MUL2662,33279,MUL2662,attraction & hotel,"['attraction', 'hotel']",17,18-MUL2662,SYSTEM,"Postal is cb58as, I do not have inform on the entrance fee, you may need to call them at 01223300085",postal is cb58as i do not have inform on the entrance fee you may need to call them at 01223300085,s-61,u-47
33280,18-MUL2662,33280,MUL2662,attraction & hotel,"['attraction', 'hotel']",18,19-MUL2662,USER,"Ok perfect, that is all that I needed. Thank you for your help! Goodbye.",ok perfect that is all that i needed thank you for your help goodbye,u-47,s-41


# Cluster Flow in dialogue

flow of clusters in dialogues

In [23]:
df_flow = df.groupby(['dialogue_id'], as_index = False).agg({'cluster_id': ' '.join})
df_flow

,dialogue_id,cluster_id
0,MUL0002,u-25 s-70 u-23 s-23 u-15 s-27 u-16 s-58 u-21 s-25 u-14 s-26 u-47 s-41
1,MUL0005,u-25 s-23 u-61 s-3 u-14 s-26 u-16 s-28 u-14 s-29 u-56 s-54 u-9 s-1 u-9 s-1 u-47 s-33
2,MUL0009,u-16 s-28 u-16 s-29 u-18 s-13 u-11 s-3 u-25 s-58 u-11 s-10 u-11 s-22 u-47 s-47 u-47 s-49
3,MUL0010,u-25 s-23 u-25 s-23 u-23 s-23 u-56 s-52 u-14 s-28 u-43 s-26 u-15 s-13 u-56 s-54 u-9 s-1 u-47 s-41
4,MUL0015,u-20 s-25 u-38 s-26 u-39 s-14 u-11 s-3 u-25 s-23 u-22 s-23 u-57 s-54 u-9 s-1 u-47 s-41
...,...,...
2501,WOZ20636,u-23 s-23 u-42 s-23 u-46 s-51
2502,WOZ20642,u-25 s-23 u-48 s-52 u-56 s-54 u-46 s-38
2503,WOZ20646,u-25 s-62 u-57 s-52 u-56 s-54 u-46 s-41 u-55 s-43
2504,WOZ20670,u-25 s-67 u-25 s-23 u-56 s-55 u-46 s-41


Sorted count of repeating flows

In [24]:
flow_count = df_flow.groupby(['cluster_id'])['dialogue_id'].count().to_frame()
flow_count = flow_count.sort_values(['dialogue_id'], ascending=False).reset_index()

flow_count.columns = ["flow", "count"]
flow_count.head(10)

,flow,count
0,u-9 s-1 u-47 s-41,4
1,u-30 s-52 u-56 s-57 u-47 s-41,3
2,u-9 s-10 u-8 s-1 u-47 s-41,3
3,u-30 s-52 u-56 s-54 u-47 s-41,3
4,u-25 s-23 u-56 s-54 u-46 s-41,3
5,u-30 s-52 u-56 s-57 u-46 s-63,2
6,u-30 s-52 u-56 s-52,2
7,u-30 s-52 u-56 s-67 u-46 s-33,2
8,u-31 s-53 u-56 s-54 u-47 s-41,2
9,u-9 s-9 u-15 s-1 u-47 s-41,2


Getting Sub-flows from flows withn at least N turns

In [25]:
def allSubFlowsGreaterThanN(s,n):
    elements = s.split(" ")
    l = len(elements)
    if l < 5:
      return []
    elif l == 5:
      return [s]
    
    #print(elements)
    subs = []
    for i in range(0, l - n-1 ):
      for j in range(i+n, l+1):
        #print(i,j )
        subs.append("|".join(elements[i:j]))
    return subs



In [26]:
min_no_of_turns = 4
flows_substrings = []
for flow in flow_count["flow"]:
  flows_substrings += allSubFlowsGreaterThanN(flow, min_no_of_turns)

In [27]:
len(flows_substrings)

176019

Counting frequency of each sub-flows

In [28]:
frequency = collections.Counter(flows_substrings)
df_freq = pd.DataFrame(dict(frequency).items(), columns = ["sub-flow", "count"])
df_freq['length'] = df_freq.apply(lambda row: math.ceil(len(row['sub-flow'])/5), axis=1)
df_freq

,sub-flow,count,length
0,u-30|s-52|u-56|s-57,7,4
1,u-30|s-52|u-56|s-57|u-47,5,5
2,u-30|s-52|u-56|s-57|u-47|s-41,1,6
3,u-9|s-10|u-8|s-1,14,4
4,u-9|s-10|u-8|s-1|u-47,10,5
...,...,...,...
168029,s-69|u-20|s-58|u-56|s-56|u-46|s-46,1,7
168030,u-20|s-58|u-56|s-56|u-46,1,5
168031,u-20|s-58|u-56|s-56|u-46|s-46,1,6
168032,u-9|s-9|u-9|s-1|u-54,1,4


Sorted frequency of subflows

In [29]:
df_freq_sorted = df_freq.sort_values(['count', "length"], ascending=False).reset_index()
df_freq_sorted = df_freq_sorted[df_freq_sorted['count'] > 1]
df_freq_sorted.head(10)

,index,sub-flow,count,length
0,4932,u-25|s-23|u-11|s-3,67,4
1,23,u-25|s-23|u-25|s-23,58,4
2,4513,u-25|s-23|u-22|s-23,58,4
3,21169,u-25|s-23|u-23|s-23,33,4
4,2457,u-11|s-3|u-25|s-23,31,4
5,9,u-25|s-23|u-56|s-54,29,4
6,2416,u-11|s-3|u-9|s-1,29,4
7,1844,u-11|s-21|u-41|s-3,28,4
8,4531,s-23|u-22|s-23|u-11,28,4
9,5185,u-25|s-23|u-11|s-3|u-47,25,5
